In [1]:
include("JuliRay.jl")
Base.@irrational ° 0.0174532925199432957692369076848861271344 (big(pi)/big(180))

In [2]:
ε=10^(-3)

function f(x)
    ifelse(x>0,exp(-1/x),0)
end
function g(x)
    f(2x/√3)/(f(2x/√3)+f(2/√3-2x/√3))
end
function Smooth(a,b,x)
    g((x-a)/(b-a))
end

VERTEX=Int
EDGE=Array{VERTEX,1}
FACE=Array{EDGE,1}
CELL=Array{FACE,1}

function ℝ³⭢S³(p::Array{T,1}) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    end
    return [2p[1]/(1+p[1]^2+p[2]^2+p[3]^2),2p[2]/(1+p[1]^2+p[2]^2+p[3]^2),2p[3]/(1+p[1]^2+p[2]^2+p[3]^2),(-1+p[1]^2+p[2]^2+p[3]^2)/(1+p[1]^2+p[2]^2+p[3]^2)]
end

function ℝ³⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    elseif(θ<ε)
        return [p[1:3]...,0.0]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        P=p*cot((π/2-θ)/2+π/4)
        return ℝ³⭢S³(P)*r+c
    end
end

function S³⭢ℝ³(q::Array{T,1}) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(!(norm(q)≈1.0))
        print(norm(q))
        error("No Point on unit S³, radius: $(norm(q))")
    end
    return [q[1]/(1-q[4]),q[2]/(1-q[4]),q[3]/(1-q[4])]
end

function S³⭢ℝ³(q::Array{T,1},θ::Real) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(θ<ε)
        return q[1:3]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        return S³⭢ℝ³((q-c)/r)*tan((π/2-θ)/2+π/4)
    end
end

function ℝ⁴⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(θ<ε)
        return [p[1:3]...,0.0]
    else
        R=1/sin(θ)
        O=[0,0,0,cot(θ)]
        return O+R*normalize(p-O)
    end
end

function NormalVector(p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    A=hcat(p₁-p₄,p₂-p₄,p₃-p₄)
    return normalize([(-1)^i*det(A[deleteat!(collect(1:4),i),:]) for i ∈ 1:4])
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃)
    return q-2*dot(𝒏,q-p₁)*𝒏
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃,p₄)
    return q-2*dot(𝒏,q-p₁)*𝒏
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector, θ)
    𝒏=NormalVector(p₁,p₂,p₃,p₄)
    return ℝ⁴⭢S³(q-2*dot(𝒏,q-p₁)*𝒏,θ)
end

function vertices(face::FACE)
    f=copy(face)
    v=union(vcat(f...))
    w=[v[1]]
    for j ∈ 1:(length(v)-1)
        i=findfirst(e->w[end] ∈ e, f)
        push!(w,filter(v->v≠w[end],f[i])[1])
        deleteat!(f,i)
    end
    return w
end

function PickFace(cell::CELL, v::RealVector,POINTS³)
    if(norm(v)≈0)
        error("vector v must be non-zero")
    end
    faces=vertices.(cell)
    fpts=(i->POINTS³[i]).(faces)
    return cell[findmax([dot(v,+(pts...)) for pts ∈ fpts])[2]]
end

function NewCell(cell::CELL,v::RealVector,θ,POINTS³, POINTS⁴)
    n=length(POINTS³)
    c=copy(cell)
    face=PickFace(cell,v,POINTS³)
    IND_face=vertices(face)
    IND_cell=Int[]
    for f ∈ c for e ∈ f for v ∈ e push!(IND_cell,v) end end end
    IND_cell=union(IND_cell)
    IND_cell2=[
        if(i ∈ IND_face)
            i
        else
            push!(POINTS³,Mirror(POINTS³[i],POINTS³[IND_face[1]],POINTS³[IND_face[2]],POINTS³[IND_face[3]]));
            if(θ<ε)
                push!(POINTS⁴,[POINTS³[end]...,0]);
            else
                O=[0,0,0,cot(θ)]
                push!(POINTS⁴,Mirror(POINTS⁴[i],O,POINTS⁴[IND_face[1]],POINTS⁴[IND_face[2]],POINTS⁴[IND_face[3]],θ));
            end
            n=n+1
        end
        for i ∈ IND_cell]
    return [[[IND_cell2[findfirst(w->w==v,IND_cell)] for v ∈ e] for e ∈ f] for f ∈ c]
end

function SphericalSphere(v,r::Real,θ::Real) where T<:RealVector
    V=S³⭢ℝ³(v,θ)
    return Sphere(V,r)
end

function SphericalCylinder(v₁,v₂,r::Real,θ::Real) where T<:RealVector
    w₁=ℝ⁴⭢S³((v₁+v₂)/2,θ)
    V₁=S³⭢ℝ³(v₁,θ)
    V₂=S³⭢ℝ³(v₂,θ)
    W₁=S³⭢ℝ³(w₁,θ)
    return Arc(V₁,W₁,V₂,r)
end

function SphericalPolygon(v::Array{T,1},θ::Real) where T<:RealVector
    n=length(v)
    u=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    U=S³⭢ℝ³(u,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    if(θ<ε)
        return Polygon(V)
    elseif(rank(hcat(V...),atol=1.0e-12)==2)
        m=4
        vw=copy(v)
        for _ ∈ 1:m
            l=length(vw)
            vw=[ℝ⁴⭢S³((vw[(i+1)÷2]+vw[mod(i÷2,l)+1])/2,θ) for i ∈ 1:2l]
        end
        VW=(q->S³⭢ℝ³(q,θ)).(vw)
        return csgUnion(Polygon(VW))
    else
        O=Circumcenter(U,V[1],V[2],V[3])
        sphere=Sphere(O,norm(U-O))
        N=NormalVector(V[1],W[1],V[2]);
        direction=sign(dot(N,O-U))

        cylinders=csgIntersection([
                (V₁=V[i];
                V₂=V[mod(i,n)+1];
                V₃=V[mod(i+1,n)+1];
                C=Circumcenter(V₁,W[i],V₂);
                N=NormalVector(V₁,W[i],V₂);
                direction=sign(dot(N,U-W[i]));
                cylinder=Cylinder(C,C+2*direction*norm(U-O)*N,norm(U-O)))
                for i ∈ 1:n
                ])
        return csgClip(sphere,cylinders)
    end
end

function Cells2Object(cells::Array{CELL,1},θ,POINTS⁴;rᵥ=0.05,rₑ=0.025,color=RGB(1,1,1))
    cs=copy(cells)
    fs=DeleteDuplicates(vcat(cs...))
    es=DeleteDuplicates(vcat(fs...))
    vs=DeleteDuplicates(vcat(es...))
    V=rgbColor(csgUnion([SphericalSphere(POINTS⁴[v],rᵥ,θ) for v ∈ vs]),RGB(0.1,0.1,0.1))
    E=rgbColor(csgUnion([SphericalCylinder(POINTS⁴[e[1]],POINTS⁴[e[2]],rₑ,θ) for e ∈ es]),RGB(0.2,0.2,0.2))
    F=rgbftColor(csgUnion([SphericalPolygon([POINTS⁴[i] for i ∈ vertices(f)],θ) for f ∈ fs]),color,FT(0.1,0.3))
    return csgUnion(V,E,F)
end

Cells2Object (generic function with 1 method)

In [3]:
# F₄
𝒑₁=[1,1,1]
𝒑₂=[1,-1,-1]
𝒑₃=[-1,1,-1]
𝒑₄=[-1,-1,1]
POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄]
POINTS=normalize.(POINTS)
v₁=1
v₂=2
v₃=3
v₄=4
e₁=[v₁,v₂]
e₂=[v₁,v₃]
e₃=[v₁,v₄]
e₄=[v₂,v₃]
e₅=[v₂,v₄]
e₆=[v₃,v₄]

f₁=[e₄,e₅,e₆]
f₂=[e₂,e₃,e₆]
f₃=[e₁,e₃,e₅]
f₄=[e₁,e₂,e₄]
c₁=[f₁,f₂,f₃,f₄]
const F₄=normalize.(POINTS), c₁

# F₆
𝒑₁=[1,1,1]
𝒑₂=[1,1,-1]
𝒑₃=[1,-1,1]
𝒑₄=[1,-1,-1]
𝒑₅=[-1,1,1]
𝒑₆=[-1,1,-1]
𝒑₇=[-1,-1,1]
𝒑₈=[-1,-1,-1]
POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄,𝒑₅,𝒑₆,𝒑₇,𝒑₈]
POINTS=normalize.(POINTS)
v₁=1
v₂=2
v₃=3
v₄=4
v₅=5
v₆=6
v₇=7
v₈=8
e₁=[v₁,v₂]
e₂=[v₁,v₃]
e₃=[v₁,v₅]
e₄=[v₂,v₄]
e₅=[v₂,v₆]
e₆=[v₃,v₄]
e₇=[v₃,v₇]
e₈=[v₄,v₈]
e₉=[v₅,v₆]
e₁₀=[v₅,v₇]
e₁₁=[v₆,v₈]
e₁₂=[v₇,v₈]
f₁=[e₁,e₂,e₄,e₆]
f₂=[e₁,e₃,e₅,e₉]
f₃=[e₂,e₃,e₇,e₁₀]
f₄=[e₄,e₅,e₈,e₁₁]
f₅=[e₆,e₇,e₈,e₁₂]
f₆=[e₉,e₁₀,e₁₁,e₁₂]
c₁=[f₁,f₂,f₃,f₄,f₅,f₆]
const F₆=normalize.(POINTS), c₁

(Array{Float64,1}[[0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [0.5773502691896258, 0.5773502691896258, -0.5773502691896258], [0.5773502691896258, -0.5773502691896258, 0.5773502691896258], [0.5773502691896258, -0.5773502691896258, -0.5773502691896258], [-0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [-0.5773502691896258, 0.5773502691896258, -0.5773502691896258], [-0.5773502691896258, -0.5773502691896258, 0.5773502691896258], [-0.5773502691896258, -0.5773502691896258, -0.5773502691896258]], Array{Array{Int64,1},1}[[[1, 2], [1, 3], [2, 4], [3, 4]], [[1, 2], [1, 5], [2, 6], [5, 6]], [[1, 3], [1, 5], [3, 7], [5, 7]], [[2, 4], [2, 6], [4, 8], [6, 8]], [[3, 4], [3, 7], [4, 8], [7, 8]], [[5, 6], [5, 7], [6, 8], [7, 8]]])

In [4]:
function C₈(θ)
    h=1/2
    r=√(1-h^2)

    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]

    POINTS, c₁=F₆
    POINTS=r*copy(POINTS)
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]
    c₂=NewCell(c₁,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₁,[1,0,0],θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,[0,0,-1],θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₆,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₅,[0,1,0],θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈]
    return cells, POINTS³, POINTS⁴
end

C₈ (generic function with 1 method)

In [5]:
M=120
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=C₈(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1),rₑ=0.012,rᵥ=0.03),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.15,width=1200,height=900),name="8-Cell",index=i+1)
end

InterruptException: InterruptException:

In [78]:
function C₁₆(θ)
    h=1/2
    r=√(1-h^2)

    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]

    POINTS, c₁=F₄
    POINTS=r*copy(POINTS)
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]
    V=[1,-2,0]
    c₂=NewCell(c₁,V,θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,V,θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₃,V,θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,-V,θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,-V,θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₆,-V,θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₇,-V,θ,POINTS³, POINTS⁴)
    c₉=NewCell(c₁,[-2,-1,0],θ,POINTS³, POINTS⁴)
    W=[-1,-0.5,1]
    c₁₀=NewCell(c₉,W,θ,POINTS³, POINTS⁴)
    c₁₁=NewCell(c₁₀,W,θ,POINTS³, POINTS⁴)
    c₁₂=NewCell(c₁₁,W,θ,POINTS³, POINTS⁴)
    c₁₃=NewCell(c₉,-W,θ,POINTS³, POINTS⁴)
    c₁₄=NewCell(c₁₃,-W,θ,POINTS³, POINTS⁴)
    c₁₅=NewCell(c₁₄,-W,θ,POINTS³, POINTS⁴)
    c₁₆=NewCell(c₁₅,-W,θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈,c₉,c₁₀,c₁₁,c₁₂,c₁₃,c₁₄,c₁₅,c₁₆]
    # cells=[c₉,c₁₀,c₁₁,c₁₂,c₁₃,c₁₄,c₁₅,c₁₆]
    # cells=[c₉,c₁₀,c₁₁]
    return cells, POINTS³, POINTS⁴
end

C₁₆ (generic function with 1 method)

In [81]:
M=120
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=C₁₆(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0.2,1,1)),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.12,width=600,height=450),name="16-Cell",index=i+1)
end

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           59580   Smpls/Pxl: 0.21
Rays:             468266   Saved:               780   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  1136736           54978      4.84
Polygon                        2752263          437201     15.89
Sphere                            7765            2986     38.45
Bounding Box                  21813172         7809312     35.80
----------------------------------------------------------------------------
Shadow Ray Tests:            331796   Succeeded:                

Parser Options
  Input file: 16-Cell_000002.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....600 by 450 (rows 1 to 450, columns 1 to 600).
  Output file..........16-Cell_000002.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           58950   Smpls/Pxl: 0.21
Rays:             467808   Saved:              1284   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Cone/Cylinder                  1142648           54329      4.75
Polygon                        2794978          444583     15.91
Sphere                            8019            3109     38.77
Bounding Box                  22059873         7946972     36.02
----------------------------------------------------------------------------
Shadow Ray Tests:            336230   Succeeded:                

File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File '16-Cell_000003.pov' li

...

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           62784   Smpls/Pxl: 0.22
Rays:             476291   Saved:               260   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           47822666        18965222     39.66
CSG Intersection              47822666         4819334     10.08
Sphere                        39829897        36152667     90.77
Torus                         23911333          234790      0.98
Torus Bound                   23911333          361380      1.51
Clipping Object               46849804          679034      1.45
Bounding Bo

Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           60381   Smpls/Pxl: 0.21
Rays:             472804   Saved:               301   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           47362906        18915612     39.94
CSG Intersection              47362906         4861725     10.26
Sphere                        39536892        35835632     90.64
Torus                         23681453          224506      0.95
Torus Bound                   23681453          354937      1.50
Clipping Object               46714893          677885      1.45
Bounding Bo

----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          109
Infinite Objects:         10
Light Sources:             1
Total:                   120
----------------------------------------------------------------------------
Parser Time
  Parse Time:       0 hours  0 minutes  0 seconds (0.005 seconds)
              using 1 thread(s) with 0.004 CPU-seconds total
  Bounding Time:    0 hours  0 minutes  0 seconds (0.000 seconds)
              using 1 thread(s) with 0.000 CPU-seconds total
----------------------------------------------------------------------------
Render Options
  Quality:  9
  Bounding boxes.......On   Bounding threshold: 3
  Antialiasing.........On  (Method 1, Threshold 0.300, Depth 3, Jitter 1.00,
 Gamma 2.50)
==== [Rendering...] ========================================================


Rendered 270000 of 270000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           61767   Smpls/Pxl: 0.21
Rays:             477092   Saved:               379   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           47748842        19127414     40.06
CSG Intersection              47748842         4879590     10.22
Sphere                        40073340        36118512     90.13
Torus                         23874421          222275      0.93
Torus Bound                   23874421          355261      1.49
Clipping Object               46898388          685336      1.46
Bounding Bo

Rendered 270000 of 270000 pixels (100%) bytes.Rendered 130304 of 270000 pixels (48%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 600 x 450
----------------------------------------------------------------------------
Pixels:           287550   Samples:           61677   Smpls/Pxl: 0.21
Rays:             478038   Saved:               447   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           46954242        19085980     40.65
CSG Intersection              46954242         4962990     10.57
Sphere                        40228384        36156353     89.88
Torus                         23477121          218336      0.93
Torus Bound                   23477121          352128      1.50
Clipping Object               4